# 코스피50 주가 예측
데이터 출처:
http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201010106

## (2) 분류모델

In [ ]:
!pip install catboost

In [ ]:
import pandas as pd
import numpy as np
import datetime

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.metrics import accuracy_score,recall_score,confusion_matrix,classification_report
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier, XGBRFClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso, Ridge
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

from sklearn.model_selection import GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.model_selection import KFold, StratifiedKFold

from sklearn.ensemble import StackingClassifier

In [ ]:
pd.set_option('display.max_columns',None)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/데이터/주가예측/1210df.csv')

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
def make_code(x):
    x = str(x)
    return '0'*(6-len(x)) + x

In [ ]:
df['code'] = df['code'].apply(make_code)

In [ ]:
df.head()

,code,date,increase,Open1,High1,Low1,Close1,Volume1,Change1,Open3,High3,Low3,Close3,Volume3,Change3,Open7,High7,Low7,Close7,Volume7,Change7,diff_se1,diff_se3,diff_se7,diff_lh1,diff_lh3,diff_lh7,close_variation,volume_variation,kospi_close1,USD/KRW1,JPY/KRW1,BTC/KRW1
0,005930,2020-01-02,0,56200.0,56600.0,55700.0,55800.0,8356767.0,-0.012389,54700.0,55400.0,54400.0,55400.0,9645034.0,0.007273,57000.0,57300.0,55500.0,56000.0,14180520.0,-0.005329,400.0,-700.0,1000.0,-900.0,-1000.0,-1800.0,50.000000,993946.357143,2197.669922,1153.750000,10.615540,8315985.5
1,005930,2020-01-03,1,55500.0,56000.0,55000.0,55200.0,12993228.0,-0.010753,55700.0,56900.0,55500.0,56500.0,12313056.0,0.019856,56100.0,56500.0,55600.0,56000.0,12095519.0,0.000000,300.0,-800.0,100.0,-1000.0,-1400.0,-900.0,96.428571,-136013.321429,2175.169922,1153.969971,10.615074,8084144.5
2,005930,2020-01-06,0,56000.0,56600.0,54900.0,55500.0,15422255.0,0.005435,56200.0,56600.0,55700.0,55800.0,8356767.0,-0.012389,56100.0,56400.0,55100.0,55500.0,9839252.0,-0.008929,500.0,400.0,600.0,-1700.0,-900.0,-1300.0,-10.714286,-744625.964286,2176.459961,1157.150024,10.659966,8641188.0
3,005930,2020-01-07,1,54900.0,55600.0,54600.0,55500.0,10278951.0,0.000000,55500.0,56000.0,55000.0,55200.0,12993228.0,-0.010753,55600.0,55700.0,54800.0,55000.0,11868463.0,-0.009009,-600.0,300.0,600.0,-1000.0,-1000.0,-900.0,-78.571429,186902.964286,2155.070068,1165.849976,10.798657,9069786.0
4,005930,2020-01-08,1,55700.0,56400.0,55600.0,55800.0,10009778.0,0.005405,56000.0,56600.0,54900.0,55500.0,15422255.0,0.005435,54700.0,55400.0,54400.0,55400.0,9645034.0,0.007273,-100.0,500.0,-700.0,-800.0,-1700.0,-1000.0,-60.714286,-258435.607143,2175.540039,1167.420044,10.756629,9530784.0


In [ ]:
df.describe()

,increase,Open1,High1,Low1,Close1,Volume1,Change1,Open3,High3,Low3,Close3,Volume3,Change3,Open7,High7,Low7,Close7,Volume7,Change7,diff_se1,diff_se3,diff_se7,diff_lh1,diff_lh3,diff_lh7,close_variation,volume_variation,kospi_close1,USD/KRW1,JPY/KRW1,BTC/KRW1
count,33172.000000,3.317200e+04,3.317200e+04,3.317200e+04,3.317200e+04,3.317200e+04,33172.000000,3.317200e+04,3.317200e+04,3.317200e+04,3.317200e+04,3.317200e+04,33172.000000,3.317200e+04,3.317200e+04,3.317200e+04,3.317200e+04,3.317200e+04,33172.000000,33172.000000,33172.000000,33172.000000,33172.000000,33172.000000,33172.000000,33172.000000,3.317200e+04,33172.000000,33172.000000,33172.000000,3.317200e+04
mean,0.464006,2.017565e+05,2.049055e+05,1.985850e+05,2.017048e+05,1.428931e+06,0.000363,2.017371e+05,2.048848e+05,1.985628e+05,2.016802e+05,1.429929e+06,0.000322,2.017228e+05,2.048668e+05,1.985432e+05,2.016610e+05,1.435526e+06,0.000338,51.703033,56.861932,61.814060,-6320.494182,-6322.067617,-6323.643736,-6.955660,1.142009e+03,2634.095052,1204.466224,10.423717,3.541337e+07
std,0.498710,2.476302e+05,2.509438e+05,2.440305e+05,2.472631e+05,3.409891e+06,0.025825,2.476088e+05,2.509211e+05,2.440065e+05,2.472369e+05,3.412104e+06,0.025843,2.475865e+05,2.508881e+05,2.439864e+05,2.472114e+05,3.427107e+06,0.026004,7451.026571,7456.067058,7465.849632,8760.732535,8768.530856,8779.007502,3089.223054,3.538602e+05,419.994128,82.620474,0.550093,1.965977e+07
min,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,2.190000e+03,0.000000e+00,-0.248705,0.000000e+00,0.000000e+00,0.000000e+00,2.190000e+03,0.000000e+00,-0.248705,0.000000e+00,0.000000e+00,0.000000e+00,2.190000e+03,0.000000e+00,-0.248705,-196000.000000,-196000.000000,-196000.000000,-197000.000000,-197000.000000,-197000.000000,-39928.571429,-8.813652e+06,1457.640015,1083.680054,9.347962,6.019475e+06
25%,0.000000,5.752250e+04,5.840000e+04,5.677400e+04,5.780000e+04,2.115848e+05,-0.013158,5.758250e+04,5.840000e+04,5.678225e+04,5.780000e+04,2.117002e+05,-0.013217,5.750000e+04,5.840000e+04,5.677400e+04,5.780000e+04,2.119492e+05,-0.013158,-1000.000000,-1000.000000,-1000.000000,-7500.000000,-7500.000000,-7500.000000,-571.428571,-2.855215e+04,2327.889893,1143.002502,9.972013,1.408982e+07
50%,0.000000,1.225000e+05,1.245000e+05,1.210000e+05,1.225000e+05,4.863900e+05,0.000000,1.230000e+05,1.250000e+05,1.210000e+05,1.230000e+05,4.869240e+05,0.000000,1.230000e+05,1.250000e+05,1.210000e+05,1.230000e+05,4.886070e+05,0.000000,40.000000,40.000000,50.000000,-3500.000000,-3500.000000,-3500.000000,7.142857,-1.840179e+02,2639.060059,1188.250000,10.423645,3.423692e+07
75%,1.000000,2.260000e+05,2.300000e+05,2.225000e+05,2.255550e+05,1.312620e+06,0.011641,2.260000e+05,2.300000e+05,2.225000e+05,2.260000e+05,1.313186e+06,0.011616,2.265000e+05,2.300000e+05,2.230000e+05,2.265000e+05,1.317352e+06,0.011569,1500.000000,1500.000000,1500.000000,-1300.000000,-1300.000000,-1294.000000,630.357143,2.777620e+04,3013.699951,1233.390015,10.756629,5.225417e+07
max,1.000000,1.770000e+06,1.784000e+06,1.756000e+06,1.770000e+06,9.030618e+07,0.299238,1.770000e+06,1.784000e+06,1.756000e+06,1.770000e+06,9.030618e+07,0.299238,1.770000e+06,1.784000e+06,1.756000e+06,1.770000e+06,9.030618e+07,0.300000,98000.000000,98000.000000,98000.000000,0.000000,0.000000,0.000000,40142.857143,9.232818e+06,3305.209961,1443.959961,11.634735,8.113841e+07


In [ ]:
y = df.loc[:,'increase']
X = df.iloc[:, 3:]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,random_state=0)
print(X_train.shape)
print(X_test.shape)

(24879, 30)
(8293, 30)


### 모형선택

In [ ]:
kfold = KFold(n_splits=5, shuffle=True, random_state=0)

In [ ]:
def classification_model(model):
    # Obtain scores of cross-validation using 10 splits and mean squared error
    scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=kfold)
    # Return mean score
    return scores.mean()

In [ ]:
classification_model(LogisticRegression())

0.5341053175847081

In [ ]:
classification_model(RandomForestClassifier())

0.6150972789994991

In [ ]:
classification_model(DecisionTreeClassifier())

0.601189605584191

In [ ]:
classification_model(XGBClassifier(booster='gbtree', verbosity = 0))

0.6160622970156246

In [ ]:
classification_model(XGBClassifier(booster='dart', verbosity = 0 ))

0.6160622970156246

In [ ]:
classification_model(XGBClassifier(booster='gblinear', verbosity = 0))

0.5357129861526281

In [ ]:
classification_model(LGBMClassifier())

0.6569398681510445

In [ ]:
classification_model(LGBMClassifier(boosting_type='goss'))

0.6081840795617961

In [ ]:
classification_model(CatBoostClassifier())

### 초모수 조절

#### LGBM

grid_search

In [ ]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_train, y_train, test_size = 0.3)

In [ ]:
def n_estimators(model):
  eval_set = [(X_test_2, y_test_2)]
  model.fit(X_train_2, y_train_2, eval_set = eval_set, early_stopping_rounds = 10)
  y_pred = model.predict(X_test_2)
  accuracy = accuracy_score(y_test_2, y_pred)
  return accuracy

In [ ]:
n_estimators(LGBMClassifier(random_state=1, n = 5000))

In [ ]:
def grid_search(model, params):
    grid_class = GridSearchCV(model, params, scoring='accuracy', cv=5, n_jobs=-1)
    
    grid_class.fit(X_train, y_train)

    best_params = grid_class.best_params_
    best_score=grid_class.best_score_

    print("Best params:", best_params) 
    print('bestscore',best_score.round(2))

In [ ]:
model=LGBMClassifier(random_state=1)
parms={'learning_rate':[0.5, 0.2, 0.1, 0.05]}
grid_search(model=model,params=parms)

Best params: {'learning_rate': 0.1}
bestscore 0.66


In [ ]:
model=LGBMClassifier(random_state=1)
parms={'max_depth':[3, 4, 5, 6]}
grid_search(model=model,params=parms)

Best params: {'max_depth': 6}
bestscore 0.66


In [ ]:
model=LGBMClassifier(random_state=1)
parms={'min_child_weight':[1, 2, 3, 4]}
grid_search(model=model,params=parms)

Best params: {'min_child_weight': 3}
bestscore 0.66


In [ ]:
model=LGBMClassifier(random_state=1)
parms={'subsample':[0.2, 0.4, 0.6]}
grid_search(model=model,params=parms)

Best params: {'subsample': 0.2}
bestscore 0.66


In [ ]:
model=LGBMClassifier(random_state=1)
parms={'colsample_bytree':[0.4, 0.6, 0.8]}
grid_search(model=model,params=parms)

Best params: {'colsample_bytree': 0.6}
bestscore 0.66


random_search

In [ ]:
kfold = KFold(n_splits=3, shuffle=True, random_state=0)

In [ ]:
def randomized_search(model,params, cv=10,runs=50):

    rand_class = RandomizedSearchCV(model, params, n_iter=runs, scoring='accuracy', 
                                  cv=kfold, n_jobs=-1, random_state=2)
    rand_class.fit(X_train, y_train)

    best_model = rand_class.best_estimator_
    best_params = rand_class.best_params_
    best_score = rand_class.best_score_

    print("Best params:", best_params)
    print("Training score: {:.3f}".format(best_score))
    return best_model

In [ ]:
model1=LGBMClassifier(random_state=2, n_jobs=-1)
params={'n_estimators':[100,200,300],'learning_rate':[0.1, 0.2, 0.05],'min_child_weight':[2,3,4],'max_depth':[5,6,7],
        'subsample':[0.1, 0.2, 0.3],'colsample_bytree':[0.5, 0.6, 0.7] }

best_model=randomized_search(model1, params)

y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Test score: {:.3f}'.format(accuracy))

Best params: {'subsample': 0.3, 'n_estimators': 300, 'min_child_weight': 4, 'max_depth': 5, 'learning_rate': 0.1, 'colsample_bytree': 0.7}
Training score: 0.664
Test score: 0.661


In [ ]:
model1=LGBMClassifier(random_state=2, n_jobs=-1)
params={'n_estimators':[250,300, 400],'learning_rate':[0.1, 0.12, 0.08],'min_child_weight':[3,4,5],'max_depth':[4, 5,6,7],
        'subsample':[0.2, 0.3, 0.4],'colsample_bytree':[0.6, 0.7, 0.8] }

best_model=randomized_search(model1, params)

y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Test score: {:.3f}'.format(accuracy))

Best params: {'subsample': 0.2, 'n_estimators': 300, 'min_child_weight': 5, 'max_depth': 7, 'learning_rate': 0.08, 'colsample_bytree': 0.6}
Training score: 0.659
Test score: 0.666


In [ ]:
model1=LGBMClassifier(random_state=2, n_jobs=-1)
params={'n_estimators':[300],'learning_rate':[0.1, 0.08, 0.06],'min_child_weight':[4,5,6],'max_depth':[6, 7,8],
        'subsample':[0.1, 0.2, 0.3],'colsample_bytree':[0.6, 0.7, 0.8] }

best_model=randomized_search(model1, params)

y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Test score: {:.3f}'.format(accuracy))

Best params: {'subsample': 0.2, 'n_estimators': 300, 'min_child_weight': 6, 'max_depth': 7, 'learning_rate': 0.06, 'colsample_bytree': 0.7}
Training score: 0.660
Test score: 0.668


최종 성능

In [ ]:
final_model = LGBMClassifier(subsample=0.2, n_estimators= 300, min_child_weight= 6, max_depth=7, 
                             learning_rate= 0.06, colsample_bytree=0.7)
final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Test score: {:.3f}'.format(accuracy))

#scores = cross_val_score(final_model, X_train, y_train, scoring='accuracy', cv=kfold)
#print("accuracy: %.3f, +/-%.3f" %(np.mean(scores),np.std(scores)))

Test score: 0.669


#### Cat

In [ ]:
n_estimators(CatBoostClassifier(random_state=1, n_estimators = 5000))

grid_search

In [ ]:
model=CatBoostClassifier(random_state=1)
parms={'learning_rate':[0.5, 0.2, 0.1, 0.05]}
grid_search(model=model,params=parms)

In [ ]:
model=CatBoostClassifier(random_state=1)
parms={'max_depth':[3, 4, 5, 6]}
grid_search(model=model,params=parms)

In [ ]:
model=CatBoostClassifier(random_state=1)
parms={'colsample_bylevel':[0.4, 0.6, 0.8]}
grid_search(model=model,params=parms)

In [ ]:
model=CatBoostClassifier(random_state=1)
parms={'subsample':[0.2, 0.4, 0.6]}
grid_search(model=model,params=parms)

In [ ]:
model=CatBoostClassifier(random_state=1)
parms={'min_child_samples':[1, 2, 3, 4]}
grid_search(model=model,params=parms)

random_search

In [ ]:
model2=CatBoostClassifier(random_state=2, n_estimators= 300)
params={'learning_rate':[0.07, 0.03, 0.05],'min_child_samples':[1,2,3],'max_depth':[6,7,8],
        'subsample':[0.5, 0.6, 0.7],'colsample_bylevel':[0.8, 1, 0.7] }

best_model=randomized_search(model2, params)

y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Test score: {:.3f}'.format(accuracy))

최종 성능

In [ ]:
final_model = CatBoostClassifier(subsample=0.7, n_estimators= 8000, min_child_samples= 1, max_depth=8, 
                                 learning_rate= 0.07, colsample_bylevel=1)
final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Test score: {:.3f}'.format(accuracy))

#scores = cross_val_score(final_model, X_train, y_train, scoring='accuracy', cv=kfold)
#print("accuracy: %.3f, +/-%.3f" %(np.mean(scores),np.std(scores)))

#### XG

grid_search

In [ ]:
model=XGBClassifier(random_state=1)
parms={'learning_rate':[0.5, 0.2, 0.1, 0.05]}
grid_search(model=model,params=parms)

Best params: {'learning_rate': 0.5}
bestscore 0.65


In [ ]:
model=XGBClassifier(random_state=1)
parms={'max_depth':[3, 4, 5, 6]}
grid_search(model=model,params=parms)

Best params: {'max_depth': 6}
bestscore 0.65


In [ ]:
model=XGBClassifier(random_state=1)
parms={'min_child_weight':[1, 2, 3, 4]}
grid_search(model=model,params=parms)

Best params: {'min_child_weight': 4}
bestscore 0.62


In [ ]:
model=XGBClassifier(random_state=1)
parms={'subsample':[0.2, 0.4, 0.6]}
grid_search(model=model,params=parms)

Best params: {'subsample': 0.6}
bestscore 0.62


In [ ]:
model=XGBClassifier(random_state=1)
parms={'colsample_bytree':[0.4, 0.6, 0.8]}
grid_search(model=model,params=parms)

Best params: {'colsample_bytree': 0.6}
bestscore 0.62


random_search

In [ ]:
model3=XGBClassifier(random_state=2, n_jobs=-1, n_estimators = 300)
params={'learning_rate':[0.5, 0.4, 0.6],'min_child_weight':[3,4,5],'max_depth':[6,7,8],
        'subsample':[0.6, 0.7, 0.8],'colsample_bytree':[0.5, 0.6, 0.7] }

best_model=randomized_search(model3, params)

y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Test score: {:.3f}'.format(accuracy))

Best params: {'subsample': 0.8, 'min_child_weight': 5, 'max_depth': 6, 'learning_rate': 0.4, 'colsample_bytree': 0.5}
Training score: 0.628
Test score: 0.633


In [ ]:
final_model = XGBClassifier(subsample=0.8, n_estimators= 300, min_child_weight= 5, max_depth=6, 
                            learning_rate= 0.4, colsample_bytree=0.5)
final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Test score: {:.3f}'.format(accuracy))

Test score: 0.635


#### rf

grid_search

In [ ]:
def grid_search(model, params):
    grid_class = GridSearchCV(model, params, scoring='accuracy', cv=5, n_jobs=-1)
    
    grid_class.fit(X_train, y_train)

    best_params = grid_class.best_params_
    best_score=grid_class.best_score_

    print("Best params:", best_params) 
    print('bestscore',best_score.round(2))

In [ ]:
model=RandomForestClassifier(random_state=1)
parms={'max_depth':[3, 4, 5, 6]}
grid_search(model=model,params=parms)

Best params: {'max_depth': 6}
bestscore 0.56


In [ ]:
model=RandomForestClassifier(random_state=1)
parms={'min_samples_split':[6,7,8]}
grid_search(model=model,params=parms)

Best params: {'min_samples_split': 8}
bestscore 0.62


In [ ]:
model=RandomForestClassifier(random_state=1)
parms={'max_features':[0.2, 0.4, 0.6]}
grid_search(model=model,params=parms)

Best params: {'max_features': 0.6}
bestscore 0.67


random_search

In [ ]:
model4=RandomForestClassifier(random_state=2, n_jobs=-1 , n_estimators = 300)
params={'max_depth':[6,7,8], 'min_samples_split':[7,8,9],'max_features':[0.5, 0.6, 0.7] }

best_model=randomized_search(model4, params)

y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Test score: {:.3f}'.format(accuracy))

Best params: {'min_samples_split': 7, 'max_features': 0.7, 'max_depth': 8}
Training score: 0.592
Test score: 0.599


최종 성능

In [ ]:
final_model = RandomForestClassifier(n_estimators= 300, min_samples_split= 7, max_depth=8, max_features=0.7)
final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Test score: {:.3f}'.format(accuracy))

#scores = cross_val_score(final_model, X_train, y_train, scoring='accuracy', cv=kfold)
#print("accuracy: %.3f, +/-%.3f" %(np.mean(scores),np.std(scores)))

Test score: 0.595
